# we only use ACS census data from 2008 - 2020

In [1]:
import os
import numpy as np
import astropy.table as aTable

In [2]:
fema = aTable.Table.read('/Users/chahah/data/noah/fema.zipcode.fsf.acs.rainfall.csv', format='csv')
print('%i entries' % len(fema))

127535 entries


In [3]:
fema[:5]

reportedZipcode,yearOfLoss,monthOfLoss,state,longitude,latitude,crs_treat,communityRatingSystemDiscount,primaryResidence,policyCount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,amountPaidOnTotalClaim,totalTotalInsuranceCoverage,zipcode,count_property,count_fema_sfha,pct_fema_sfha,count_fs_risk_2020_5,pct_fs_risk_2020_5,count_fs_risk_2050_5,pct_fs_risk_2050_5,count_fs_risk_2020_100,pct_fs_risk_2020_100,count_fs_risk_2050_100,pct_fs_risk_2050_100,count_fs_risk_2020_500,pct_fs_risk_2020_500,count_fs_risk_2050_500,pct_fs_risk_2050_500,pct_fs_fema_difference_2020,avg_risk_score_all,avg_risk_score_2_10,avg_risk_fsf_2020_100,avg_risk_fsf_2020_500,avg_risk_score_sfha,avg_risk_score_no_sfha,count_floodfactor1,count_floodfactor2,count_floodfactor3,count_floodfactor4,count_floodfactor5,count_floodfactor6,count_floodfactor7,count_floodfactor8,count_floodfactor9,count_floodfactor10,2018.population,2018.median_household_income,2018.renter_fraction,2018.educated_fraction,2014.population,2014.median_household_income,2014.renter_fraction,2014.educated_fraction,2010.population,2010.median_household_income,2010.renter_fraction,2010.educated_fraction,AREA,mean_rainfall,sum_rainfall,median_rainfall
int64,int64,int64,str2,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64
1001,2005,10,MA,-72.6,42.1,0.0,11.0,0.6666666666666666,3.0,5958.55,0.0,455800,26400,5958.55,482200,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,17423,56714,0.21735636801928485,0.27717140661029976,17380,63682,0.18601841196777905,0.24933451641526175,0.005208333333341666,321.2156677246094,963.6470031738281,312.2799987792969
1001,2011,4,MA,-72.6,42.0,0.0,11.0,1.0,1.0,10545.77,1473.46,250000,100000,12019.23,350000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,17423,56714,0.21735636801928485,0.27717140661029976,17380,63682,0.18601841196777905,0.24933451641526175,0.005208333333341666,147.1486612955729,441.44598388671875,147.62399291992188
1001,2011,8,MA,-72.6,42.1,0.0,11.0,1.0,2.0,9320.56,0.0,262000,5000,9320.56,267000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,17423,56714,0.21735636801928485,0.27717140661029976,17380,63682,0.18601841196777905,0.24933451641526175,0.005208333333341666,266.2306620279948,798.6919860839844,266.02899169921875
1001,2014,8,MA,-72.6,42.1,0.0,11.0,1.0,1.0,590.26,0.0,250000,100000,590.26,350000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,17423,56714,0.21735636801928485,0.27717140661029976,17380,63682,0.18601841196777905,0.24933451641526175,0.005208333333341666,99.75433349609375,299.26300048828125,101.04000091552734
1002,1991,8,MA,-70.6,41.7,0.0,11.0,0.0,1.0,33851.0,0.0,70000,0,33851.0,70000,1002,7048,80,1.1,194,2.8,204,2.9,497,7.1,513,7.3,654,9.3,670,9.5,5.9,1.53,6.6,7.61,6.7,5.9,1.48,6378,15,53,87,34,172,94,14,98,103,30099,60540,0.4102129638858434,0.7107991817732948,29970,48923,0.4468134801468135,0.65777009247425,28718,56175,0.3966153631868515,0.6823411319589221,0.01388888888891111,233.21237564086914,1865.69900

In [4]:
year2018 = (fema['yearOfLoss'] > 2016)
year2014 = (fema['yearOfLoss'] > 2012) & (fema['yearOfLoss'] <= 2016)
year2010 = (fema['yearOfLoss'] > 2003) & (fema['yearOfLoss'] <= 2012)
years = year2010 | year2014 | year2018

In [5]:
print('%i of %i entries have year > 2003' % (np.sum(years), len(years)))

74288 of 127535 entries have year > 2003


In [6]:
census_cols = []
for col in fema.columns: 
    if '2014' in col: 
        census_cols.append(col.split('.')[1])

In [7]:
for col in census_cols: 
    fema[col] = 0.
    fema[col][year2018] = fema['2018.'+col][year2018]
    fema[col][year2014] = fema['2018.'+col][year2014]
    fema[col][year2010] = fema['2018.'+col][year2010]

In [8]:
fema_post2003 = fema[years]

In [9]:
fema_post2003[::1000][:10]

reportedZipcode,yearOfLoss,monthOfLoss,state,longitude,latitude,crs_treat,communityRatingSystemDiscount,primaryResidence,policyCount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,amountPaidOnTotalClaim,totalTotalInsuranceCoverage,zipcode,count_property,count_fema_sfha,pct_fema_sfha,count_fs_risk_2020_5,pct_fs_risk_2020_5,count_fs_risk_2050_5,pct_fs_risk_2050_5,count_fs_risk_2020_100,pct_fs_risk_2020_100,count_fs_risk_2050_100,pct_fs_risk_2050_100,count_fs_risk_2020_500,pct_fs_risk_2020_500,count_fs_risk_2050_500,pct_fs_risk_2050_500,pct_fs_fema_difference_2020,avg_risk_score_all,avg_risk_score_2_10,avg_risk_fsf_2020_100,avg_risk_fsf_2020_500,avg_risk_score_sfha,avg_risk_score_no_sfha,count_floodfactor1,count_floodfactor2,count_floodfactor3,count_floodfactor4,count_floodfactor5,count_floodfactor6,count_floodfactor7,count_floodfactor8,count_floodfactor9,count_floodfactor10,2018.population,2018.median_household_income,2018.renter_fraction,2018.educated_fraction,2014.population,2014.median_household_income,2014.renter_fraction,2014.educated_fraction,2010.population,2010.median_household_income,2010.renter_fraction,2010.educated_fraction,AREA,mean_rainfall,sum_rainfall,median_rainfall,population,median_household_income,renter_fraction,educated_fraction
int64,int64,int64,str2,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1001,2005,10,MA,-72.6,42.1,0.0,11.0,0.6666666666666666,3.0,5958.55,0.0,455800,26400,5958.55,482200,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,17423,56714,0.21735636801928485,0.27717140661029976,17380,63682,0.18601841196777905,0.24933451641526175,0.005208333333341666,321.2156677246094,963.6470031738281,312.2799987792969,16064.0,66088.0,0.21358316733067728,0.3477663785443089
2301,2018,3,MA,-71.025,42.1,0.0,11.0,0.75,4.0,15672.57,0.0,950000,280000,15672.57,1230000,2301,14504,500,3.4,19,0.1,21,0.1,1067,7.4,1139,7.9,1717,11.8,1783,12.3,3.9,1.52,5.25,6.25,5.37,4.72,1.41,12721,66,183,372,111,813,195,10,21,12,63920,57977,0.4450250312891114,0.18880007943206076,60659,44873,0.49417234046060765,0.16710068325383526,60265,45017,0.4729279017671949,0.1732250891979937,0.0017361111111138887,192.7689971923828,192.7689971923828,192.7689971923828,63920.0,57977.0,0.4450250312891114,0.18880007943206076
3223,2011,8,NH,-71.70000000000002,43.89230769230768,0.0,11.0,0.15384615384615385,13.0,179782.44999999995,13200.69,472700,40000,192983.13999999996,512700,3223,2889,191,6.6,227,7.9,239,8.3,403,13.9,419,14.5,484,16.8,502,17.4,7.3,2.07,7.19,8.0,7.37,6.17,1.79,2387,17,24,31,38,102,66,13,101,110,3382,72059,0.12921348314606743,0.46470828233374134,3417,61150,0.09891717881182324,0.428997390980246,3389,47176,0.14753614635585718,0.28383381324557794,0.026041666666708332,289.333534749349,4340.003021240234,287.7439880371094,3382.0,72059.0,0.12921348314606743,0.46470828233374134
5851,2011,8,VT,-72.02105263157895,44.5,0.0,11.0,0.42105263157894735,19.0,498325.39,84901.78000000001,2130400,450200,583227.17,2580600,5851,2745,99,3.6,237,8.6,247,9.0,666,24.3,675,24.6,761,27.7,771,28.1,20.7,2.7,7.04,7.51,7.1,6.96,2.54,1974,10,28,39,26,349,49,8,87,175,6212,56160,0.25386349001931746,0.27491840321365807,6635,40629,0.2690278824415976,0.241271370093908,6505,37857,0.33512682551883166,0.24353607485840925,0.022569444444480554,235.97354008601263,3067.656021118164,233.77999877929688,6212.0,56160.0,0.25386349001931746,0.27491840321365807
6831,

In [10]:
no_income_or_number = (fema_post2003['population'] == 0) | (fema_post2003['median_household_income'] == 0)
print('%i of %i have no income or number' % (np.sum(no_income_or_number), len(fema_post2003)))

0 of 74288 have no income or number


In [11]:
fema_post2003['amountPaidOnTotalClaim_per_policy'] = fema_post2003['amountPaidOnTotalClaim'] / fema_post2003['policyCount']

In [13]:
fema_post2003.write('/Users/chahah/data/noah/zipcode.fema.fsf.acs.rainfall.csv', format='csv', overwrite=True)